### SSS movies 

In [8]:
# mom6_tools
from mom6_tools.DiagsCase import DiagsCase
from mom6_tools.MOM6grid import MOM6grid
from mom6_tools.m6toolbox import genBasinMasks
from mom6_tools.m6plot import myStats, annotateStats
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import xarray as xr
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
from xmovie import Movie
import warnings, os, yaml
warnings.filterwarnings("ignore")

In [2]:
%%capture 
# comment above line to see details about the run(s) displayed
from misc import *
%matplotlib inline

In [3]:
from ncar_jobqueue import NCARCluster
from dask.distributed import Client

cluster = NCARCluster(memory='2GB', processes=1)
cluster.scale(20)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gmarques/high-mem/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gmarques/high-mem/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.63:37936,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gmarques/high-mem/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
def preprocess(ds):
    variables = ['tos','sos']
    return ds[variables]

### SSS in the Lab Sea

In [5]:
casename

['bmom.e23.f09_t061_zstar_N65.nuopc.GM_tuning.002',
 'b.cesm3_cam058_mom_c.B1850WscMOM.ne30_L58_t061.009',
 'b.cesm3_cam058_mom_e.B1850MOM.f09_L32_t061.cam6_cice5.016',
 'b.cesm3_cam058_mom_e.B1850MOM.ne30_L32_t061.cam6_cice5.017',
 'b.cesm3_cam058_mom_e.B1850MOM.ne30_L32_t061.cam6_cice5.018b',
 'b.cesm3_cam058_mom_e.B1850MOM.ne30_L32_t061.cam6_cice5.019b',
 'b.cesm3_cam058_mom_e.B1850MOM.ne30_L32_t061.camdev_cice5.020']

In [15]:
# load hm files 
for n in [1,2,3,4,5,6]:
  print(casename[n])
  ds = xr.open_mfdataset(OUTDIR[n]+'/'+casename[n]+'.mom6.hm_*.nc', 
                       parallel=True, preprocess=preprocess)
    
  ds1 = ds.sel(xh=slice(-70,-20), yh=slice(40,75)).sel(time=slice('0001-01-01',end_date))
  mov = Movie(ds1.sos, plotmethod='pcolormesh', vmin=27, vmax=37, 
            levels=50, cmap=cm.cm.haline, extend="both")
    
  fname='sss/{}_sss_north_atlantic.mov'.format(label[n])
  mov.save(fname,progress=True, framerate=15, 
         overwrite_existing=True, parallel=True)

b.cesm3_cam058_mom_c.B1850WscMOM.ne30_L58_t061.009
Movie created at case009_sss_north_atlantic.mov
b.cesm3_cam058_mom_e.B1850MOM.f09_L32_t061.cam6_cice5.016
Movie created at case016_sss_north_atlantic.mov
b.cesm3_cam058_mom_e.B1850MOM.ne30_L32_t061.cam6_cice5.017
Movie created at case017_sss_north_atlantic.mov
b.cesm3_cam058_mom_e.B1850MOM.ne30_L32_t061.cam6_cice5.018b
Movie created at case018b_sss_north_atlantic.mov
b.cesm3_cam058_mom_e.B1850MOM.ne30_L32_t061.cam6_cice5.019b
Movie created at case019b_sss_north_atlantic.mov
b.cesm3_cam058_mom_e.B1850MOM.ne30_L32_t061.camdev_cice5.020
Movie created at case020_sss_north_atlantic.mov


In [14]:
%%time
ds = xr.open_mfdataset(OUTDIR[1]+'/'+casename[1]+'.mom6.hm_*.nc', 
                       parallel=True, preprocess=preprocess)
ds1 = ds.sel(xh=slice(-70,-20), yh=slice(40,75)).sel(time=slice('0001-01-01','0002-01-01'))
mov = Movie(ds1.sos, plotmethod='pcolormesh', vmin=27, vmax=37, 
            levels=50, cmap=cm.cm.haline, extend="both")
mov.save('movie_sss.mov',progress=True, framerate=10, 
         overwrite_existing=True, parallel=True)

#ax.set_xlim(-80,-20)
#ax.set_ylim(40,75)

Movie created at movie_sss.mov
CPU times: user 4.2 s, sys: 172 ms, total: 4.37 s
Wall time: 9.4 s


In [13]:
def custom_plotfunc(ds, fig, tt, *args, **kwargs):
    fig, ax = plt.subplots(figsize=(10, 5))
    ds.isel(time=tt).plot(ax=ax, vmin=-2, vmax=8, cmap='RdBu_r')
    ax.set_xlim(-80,-20)
    ax.set_ylim(40,75)

    return None, None
    # ^ This is not strictly necessary, but otherwise a warning will be raised.

#mov_custom.preview(30)

In [ ]:
mov_custom = Movie(ds.tos, custom_plotfunc)
mov_custom.save(
    'movie_parallel.mov',
    parallel=False,
    overwrite_existing=True,
)

In [7]:
mov_parallel = Movie(ds.tos.chunk({'time':1}).isel(time=slice(0,30)), vmin=-2, vmax=32)

In [8]:
%%time
mov_parallel.save(
    'movie_parallel.mov',
    parallel=True,
    overwrite_existing=True,
)

Movie created at movie_parallel.mov
CPU times: user 658 ms, sys: 87.8 ms, total: 746 ms
Wall time: 19.8 s
